In [115]:
from openai import OpenAI
import re
import json
DEEPSEEK_KEY=""

In [116]:
file_path="/home/tubei/Desktop/data/experiment/input/内蒙古包头市2020年中考数学试题（原卷版）.md"

# 1. 前处理
加入换行符:
1. 加粗文本**** 之后加入换行符
2. 图片文件后加入换行符

In [117]:
def add_newline_after_bold(text):
    # 定义一个正则表达式来匹配加粗文本
    regex_pattern = r"\*\*(.+?)\*\*"
    # 在匹配到的加粗文本后添加一个换行符
    modified_text = re.sub(regex_pattern, r"**\1**\n", text)
    return modified_text

def add_newline_image_item(text):
    regex_pattern = r"!\[\]\(media/(.+?)\)"
    modified_text = re.sub(regex_pattern, r"![](media/\1)\n",text)
    return modified_text



with open(file_path, 'r', encoding='utf-8') as file:
    lines=[]
    for line in file:
        processed_line = add_newline_after_bold(add_newline_image_item(line))
        # Split the processed line by newline characters and extend the list with the segments
        segments = processed_line.split('\n')
        for segment in segments:
            if segment.strip():  # This checks if the segment is not just whitespace
                lines.append(segment)

## 1.0 这边是调用大模型分割选择题的部分

In [118]:

def deepseek_align(prompt,text):

    client = OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com")
    input='input:'+text
    output='output：'
    content=prompt+input+output
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": content},
        ],
        stream=False,
        temperature=0.7
    )

    return response.choices[0].message.content


def splitAnswer(text):
    try:
        response = deepseek_align("example: input:16-20 BCACB卷二： output:{16:'B',17:'C',18:'A',19:'C',20:'B'} input:14.A 15.C 16.B 17.D 18.AB 19.ACD 20.AB非选择题部分共12题，共180分。![](media/image12.png) output:{14:'A',15:'C',16:'B',17:'D',18:'AB',19:'ACD',20:'AB'}请直接输出字典格式", text)
        
        
        pattern = r'\{[^{}]*\}'

        # 使用 `re.findall` 提取 `{}` 之间的内容
        matches = re.findall(pattern, response)
        if matches:
            json_data =matches [0]

            # 解析JSON
            data = json.loads(json_data)

            # 创建列表，每个元素为题号和答案的组合
            result = []
            for key, value in data.items():
                result.append(key+'.'+value)

            return result
        else:
            ls = list(text)
            return ls

    except Exception as e:
        print(f"An error occurred: {e}")
        ls = list(text)
        return ls
    


## 1.1 全是正则匹配行首数字的地方,重点更改用来适配

In [119]:
GET_TOPIC_PATTERN = re.compile(
    r'^>*\s*\**(\d+)\\*[．.]|'                     # 匹配数字后跟点或者全角点
    r'^>*\s*\**(\\*[（(])\d+\\*[）)]|'              # 匹配括号内的数字
    r'^>*\s*\**\\*\d+\\[．.]|'                        # 匹配数字后跟反斜杠和点
    r'^>*\s*\**\d+、|'                              # 匹配数字后跟顿号
    r'^>*\s*\**\d+[ABCD] |'                        # 匹配数字后跟ABCD中任意一个字符和空格
    r'^>*\s*\**\[(\d+)\]|'                         # 匹配方括号内的数字
    r'^>*\s*\**\d+\（|'                            # 匹配数字后跟全角开括号
    r'^>*\s*\**\d+\\*[．.]|'                            # 新增模式，匹配大于号、空格、数字及点
    r'^>*\s*\**\d+\\+|' 
    r'^>*\s*\**\s*> \s*(\d+)[．.]|'                    # 匹配大于号后跟至少一个空格，数字，和全角点
    r'^>*\s*\**([（(])\d+[）)]|'                        # 匹配括号内的数字
    r'^>*\s*\**\d+[．.]\s*|'                               # 匹配数字后跟点和空格
    r'^>*\s*\**\d+[、、]|'                                 # 匹配数字后跟顿号
    r'^>*\s*\**\d+[ABCD] |'                            # 匹配数字后跟ABCD中任意一个字符和空格
    r'^>*\s*\**\[(\d+)\]|'                             # 匹配方括号内的数字
    r'^>*\s*\**\d+（|'                                 # 匹配数字后跟全角开括号
    r'^>*\s*\**\d+\s|'                                 # 匹配数字后跟空格
    r'^>*\s*\|\s*\d+'
)




GET_TOPIC_PATTERN_IN_NOT_START = re.compile(r'(\d+)\\*[．. ]|(\\*[（(])\d+\\*[）)]|\\*\d+\\[．.]|\d+、|^\d+[ABCD]')
SIMPLE_TOPIC_PATTREN=re.compile(r"\b\d{1,3}\b")


def find_only_item(text):
    pattern_reg_item=GET_TOPIC_PATTERN 
    result_item=re.search(pattern_reg_item,text)
    return result_item


def get_topic_number(text):
    pattern_reg_item=SIMPLE_TOPIC_PATTREN
    topic=re.search(pattern_reg_item,text)
    return topic
def contain_no_Chinese(text):
    pattern = r'[\u4e00-\u9fa5]'
    result = re.sub(pattern,"",text)
    return result

## 1.2 逐行处理的函数


1. 找到选择题答案连在一起的行,拆开,check_pattern,check_proportion
2. 去除干扰的数字项,如(2分),使用正则匹配去除,remove_item_value_in_stentfront
3. 把开头的考前规则去掉,办法是找前6个数字,如果1重复了两次,则丢弃第二个1之前的全部

In [120]:
def check_pattern(text):
  
    pattern = r'\d+[．.]\s*[A-D]'
    matches = re.findall(pattern, text)
    match_count = len(matches)
    match_length = sum(len(match) for match in matches)
    total_length = len(text)

    # pattern2=r'\b[A-D]{3,6}\b'
    # matches2=re.findall(pattern2,text)
    # if matches2:
    #     return True
    if match_count >= 2 and (match_length / total_length) > 0.4:
        return  True
    else:
        return False
def check_proportion(text):
    # 正则表达式匹配数字和字母
    digits = re.findall(r'\d', text)
    letters = re.findall(r'[A-Z]', text)
    total_length = len(text)
    
    # 计算数字和字母的比例
    digit_ratio = len(digits) / total_length if total_length != 0 else 0
    letter_ratio = len(letters) / total_length if total_length != 0 else 0
    
    # 返回布尔值：是否数字比例 > 0.25 且字母比例 > 0.25
    return digit_ratio > 0.25 and letter_ratio > 0.25 and total_length>5


In [121]:

doc_length=len(lines)
process_data_sum=[]
new_lines=[]




def remove_item_value_in_stentfront(text):
    regex_pattern = r'^[\[\(\{［（｛]\s*\d+\s*分\s*[\]\)\}］）｝]'
    modified_text = re.sub(regex_pattern, '', text, count=1)
    return modified_text
        




for line in lines:
    line=remove_item_value_in_stentfront(line)
    
    if check_pattern(line) or check_proportion(line):
        print(line)

        ls=splitAnswer(line)
        new_lines.extend(ls)
    else:
        new_lines.append(line)
    



# 2. 清洗文档与找题号

得到了新的文本new_lines,接下来进行找题号(代码里也叫topic或者item)
找到的题号行如 1,2,3,4,5等连续序列

1. 先移除(1)这种,一般是小题,不是大的题号
2. 找题号的思路
    1. 首先把每行前5个字母抓下来[:5]
    2. 去除中文
    3. 抓正则匹配里的模式,行如 1. 1\ >1. **1, 等
    4. 找到后再找里面的数字部分,也就是topic
3. 为了方便后文判断答案是否存在与对齐,要确定最大题号是多少

## 2.0 清洗

In [122]:



def has_bracket(text):
    regex_pattern = r'^>*\s*\**[\[\(\{［（｛].*[\]\)\}］）｝]'
    match = re.match(regex_pattern,text)
    if match:
          return True
    else:
          return False




#找到最大题号
topic_list=[]
topic_index_list=[]
temp_max=0
for index,line in enumerate(new_lines):
      line_former_5=line[:5]
      line_wo_chinese=contain_no_Chinese(line_former_5)
      line_only_item=find_only_item(line_wo_chinese)
      if line_only_item:
            topic=get_topic_number(line_only_item.group(0))
            # if remove_wrong_topic_number(line_wo_chinese,line_only_item):
            #       continue
            if topic:
                  if has_bracket(line_wo_chinese):
                        continue
                  else:
                        topic_index_list.append([int(topic.group(0)),index])
                        topic_list.append(int(topic.group(0)))
                        if temp_max<int(topic.group(0)):
                              temp_max=int(topic.group(0))

## 2.1 判断是否有答案

接下来是判断答案是否出现,方法是看题号重叠次数
example:1,2,3,4,5,1,2,3,4
出现两次的是4个数字,前五个是题目,后四个是答案

两个判断flag,有无答案,答案是不是1,2,3开头

In [123]:
def calculate_twice_frequency(numbers):
    from collections import Counter
    
    # 计数
    count = Counter(numbers)
    
    # 筛选出现2次的数字
    twice_numbers = {num: freq for num, freq in count.items() if freq == 2}
    
    # 计算出现2次的数字占比
    if numbers:
        proportion = len(twice_numbers) / len(set(numbers))
    else:
        proportion = 0  # 如果列表为空，则比例为0

    return proportion

result = calculate_twice_frequency(topic_list)
if result>0.3:
      has_answer_flag=True
else:
      has_answer_flag=False

if has_answer_flag:
      max_index=topic_list.index(temp_max)
      ls=topic_list[max_index+1:max_index+4]
      if ls==[1,2,3]:
            answer_is_standard_flag=True
      else:
            answer_is_standard_flag=False

## 2.2 洗去考前规则

去除考前规则
考前规则,也用1,2,3 数字标注,如:

```txt
**注意事项：**

**1．本试卷共6页，满分为120分，考试时间为120分钟．**

**2．答题前，考生务必先将自己的考生号、姓名、座位号等信息填写在试卷和答题卡的指定位置，请认真核对条形码上的相关信息后，将条形码粘贴在答题卡的指定位置上．**

**3．答选择题时，必须使用2B铅笔将答题卡上对应题目的答案标号涂黑，修改时用橡皮擦干净，再选涂其他答案．**

**4．答非选择题时，必须使用0.5毫米的黑色字迹签字笔书写，作图题可先用铅笔绘出，确认后再用0.5毫米的黑色字迹签字笔描清楚．要求字体工整，笔迹清晰．严格按题号所示的答题区域作答，超出答题区域书写的答案无效；在试卷、草稿纸上答题无效．**

**5．保持答题卡清洁、完整，严禁折叠、损坏．严禁在答题卡上做任何标记，严禁使用涂改液、胶带纸、修正带．考试结来后，将本试卷和答题卡一并交回．**

**一、选择题：本大题共有12小题，每小题3分，共36分．每小题只有一个正确选项．请将答题卡上对应题目的答案标号涂黑．**

1.![](media/image2.wmf)的计算结果是（ ）
```



因此选取前6个数字,如果1出现两次以上,则说明考前规则存在,一直删到第二个1

In [124]:
from collections import Counter


## 考前规则移除
def remove_exam_rule(ls,ls_withindex):

    count = Counter(ls[:6])
    if count.get(1)>=2:
        # 找到第一个1的位置
        first_index = ls.index(1)
        # 从第一个1后面开始查找第二个1的位置
        second_index = ls.index(1, first_index + 1)
        
        return ls_withindex[:][second_index][1]
    else:
        return 0

topic_start_index=remove_exam_rule(topic_list,topic_index_list)
new_lines=new_lines[topic_start_index:]

## 2.4 找最后一题与答案之间分不开的答案area

找分不开的答案area,
1. 找到上文出现的最大题号,比如28题的index,暂定答案区域start_index
2. 从这一行collect_flag为true开始确定下一循环为end_flag
3. 在区域里一行行找,出现关键字 ["参考答案", "选择题", "单项","填空题","答题卡"],或者连续两行中文占比小于50%,则是最终答案区域start_index



In [125]:
keywords = ["参考答案", "选择题", "单项","填空题","答题卡"]

def calculate_chinese_proportion(text):
    # 统计中文字符数量
    chinese_count = sum(1 for char in text if '\u4e00' <= char <= '\u9fff')
    total_count = len(text)

    # 计算中文字符的占比
    proportion = chinese_count / total_count * 100 if total_count > 0 else 0
    return proportion

if has_answer_flag and not answer_is_standard_flag():
      not_standard_answer_area_start_index=0
      not_standard_answer_area_end_index=0
      collect_flag=False

      for index,line in enumerate(new_lines):
            line_former_5=line[:5]
            line_wo_chinese=contain_no_Chinese(line_former_5)
            line_only_item=find_only_item(line_wo_chinese)
            
            if line_only_item:

                  topic=get_topic_number(line_only_item.group(0))

                  # if remove_wrong_topic_number(line_wo_chinese,line_only_item):
                  #       continue
                  
                  if topic:
                        if has_bracket(line_wo_chinese):
                              continue
                        else:
                              if int(topic.group(0))==temp_max:
                                    if has_answer_flag and not answer_is_standard_flag:
                                          not_standard_answer_area_start_index=index
                                          collect_flag=True
                                          continue
                              if collect_flag:
                                    not_standard_answer_area_end_index=index
                                    first_answer_topic=topic
                                    break
      
      low_count = 0
      for index,line in enumerate(new_lines[not_standard_answer_area_start_index+1:not_standard_answer_area_end_index]):

            if any(keyword in line for keyword in keywords):
                  not_standard_answer_area_start_index+=index
                  break

            proportion = calculate_chinese_proportion(line)
            if proportion < 50:
                  low_count += 1
                  
            else:
                  low_count = 0
            # 检查是否连续两次占比低于50%
            if low_count >= 2:
                  not_standard_answer_area_start_index+=index
                  break
            
      prompt="请从这些文本中找出选择题或填空题1-{}的答案,输出格式类似为".format(first_answer_topic-1)
      text=''.join(new_lines[not_standard_answer_area_start_index:not_standard_answer_area_end_index])
      

In [126]:
if len(text):
    response=deepseek_align(prompt+"{'1':A,'2':B}",text)

    pattern = r'\{[^{}]*\}'

    # 使用 `re.findall` 提取 `{}` 之间的内容
    matches = re.findall(pattern, response)
    if matches:
        json_data =matches [0]

        # 解析JSON
        data = json.loads(json_data)

        # 创建列表，每个元素为题号和答案的组合
        result = []
        for key, value in data.items():
            result.append(key+'.'+value)

        # 输出结果
        print(result)
        new_lines[not_standard_answer_area_start_index:not_standard_answer_area_end_index]=result

开始最终找题号,并存为pd

In [127]:
source=file_path
process_data_sum=[]


temp_topic=0
for index,line in enumerate(new_lines):
        line_former_5=line[:5]
        line_wo_chinese=contain_no_Chinese(line_former_5)
        line_only_item=find_only_item(line_wo_chinese)
        
        if line_only_item:

            topic=get_topic_number(line_only_item.group(0))

            if topic:
                  
                  if has_bracket(line_wo_chinese):
                        continue
                  else:

                        process_data_sum.append([source,index,line,topic.group(0),line_former_5])


            else:
                  continue
            #     process_data_sum.append([source,index,line,line_only_item.group(0),line_wo_chinese])

In [128]:
import pandas as pd
columns = ['source', 'line_index', 'text','line5_item','line5_digit_origin']
df = pd.DataFrame(process_data_sum, columns=columns)
df['index_gap'] = df['line_index'].shift(-1) - df['line_index']
df.iloc[-1,5]=doc_length-df.iloc[-1,1]

In [129]:
df.head(60)

,source,line_index,text,line5_item,line5_digit_origin,index_gap
0,/home/tubei/Desktop/data/experiment/input/内蒙古包...,0,1.![](media/image2.wmf),1,1.![],3.0
1,/home/tubei/Desktop/data/experiment/input/内蒙古包...,3,2.2020年初，国家统计局发布数据，按现行国家农村贫困标准测算，截至2019年末，全国农村...,2,2.202,5.0
2,/home/tubei/Desktop/data/experiment/input/内蒙古包...,8,3.点*A*在数轴上，点*A*所对应的数用$2a + 1$表示，且点*A*到原点的距离等于3...,3,3.点*A,3.0
3,/home/tubei/Desktop/data/experiment/input/内蒙古包...,11,或1 B. ![](media/image4.wmf),1,或1 B.,1.0
4,/home/tubei/Desktop/data/experiment/input/内蒙古包...,12,或2 C.,2,或2 C.,3.0
5,/home/tubei/Desktop/data/experiment/input/内蒙古包...,15,4.下列计算结果正确的是（ ）,4,4.下列计,5.0
6,/home/tubei/Desktop/data/experiment/input/内蒙古包...,20,5.如图，$\angle ACD$是$\bigtriangleup ABC$的外角，$CE/...,5,5.如图，,4.0
7,/home/tubei/Desktop/data/experiment/input/内蒙古包...,24,6.如图，将小立方块①从6个大小相同的小立方块所搭的几何体中移走后，所得几何体（,6,6.如图，,5.0
8,/home/tubei/Desktop/data/experiment/input/内蒙古包...,29,7.两组数据：3，*a*，*b*，5与*a*，4，![](media/image8.wmf),7,7.两组数,5.0
9,/home/tubei/Desktop/data/experiment/input/内蒙古包...,34,8.如图，在$Rt \bigtriangleup ABC$中，$\angle ACB = 9...,8,8.如图，,7.0


In [130]:
ls = df['line5_item'].tolist()
int_list = [int(item.strip()) for item in ls]

In [131]:
from collections import Counter
count = Counter(int_list )
count.items()

dict_items([(1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (83, 1), (66, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)])

这里是把答案和题目分开:
思路是从最大题号往下找,1,2,3,4,1,2,3
这里1,2,3就可以配成功

In [132]:
def split_list(lst):
    import collections

    # 对列表中的元素计数
    counts = collections.Counter(lst)
    max_value = max(lst)  # 最大值
    max_count = counts[max_value]  # 最大值的计数

    # 如果最大值的计数为1，继续向下找次大的数，直到找到出现至少两次的数
    if max_count == 1:
        sorted_values = sorted(counts.items(), key=lambda x: x[0], reverse=True)
        for value, count in sorted_values:
            if count >= 2:
                first_index = lst.index(value)
                break
    else:
        first_index = lst.index(max_value)

    # 使用第一个符合条件的数字的索引来分割列表
    left = lst[:first_index + 1]
    right = lst[first_index + 1:]
    
    return left, right

# 示例列表
example_list = int_list
left, right = split_list(example_list)
print("Left part:", left)
print("Right part:", right)




Left part: [1, 2]
Right part: [3, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 83, 66, 22, 23, 24, 25, 26]


# 3. 后处理
开始看df有么有分割错,如何修补

In [133]:

def remove_wrong_topic_number(line_wo_chinese,line_only_item):
    len_out_of_number=len(line_wo_chinese)-(line_only_item.start()+len(line_only_item.group(0)))
    if len_out_of_number>2:
        return True
    else:
        return False

def regex_test(text):
    line_former_5=text[:5]
    line_wo_chinese=contain_no_Chinese(line_former_5)
    line_only_item=find_only_item(line_wo_chinese)
    topic=get_topic_number(line_only_item.group(0))
    rest=len(line_wo_chinese)-(line_only_item.start()+len(line_only_item.group(0)))
    wrongtopic_flag=remove_wrong_topic_number(line_wo_chinese,line_only_item)

    print(line_former_5,'\n',line_wo_chinese,'\n',line_only_item,'\n',topic,'\n',rest,'\n',wrongtopic_flag)



regex_test("1.![](media/image2.w")

1.![] 
 1.![] 
 <re.Match object; span=(0, 2), match='1.'> 
 <re.Match object; span=(0, 1), match='1'> 
 3 
 True
